In [113]:
import os
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

In [128]:
path = "results/"
year = "2014"
for file in os.listdir(path):
    if file == year:
        df = pd.read_csv(path+file+"/results.csv")
        print("Nam " + year +" kich thuoc :",df.shape)

Nam 2014 kich thuoc : (31984, 21)


# Boston Marathon
* Ellbow method

In [13]:
df.head()

,10k,name,division,25k,gender,age,official,bib,genderdiv,ctz,...,overall,pace,state,30k,5k,half,20k,country,city,40k
0,17.37,"Yamamoto, Hiroyuki",8,47.67,M,47,85.25,W1,8,NaN,...,8,3.27,NaN,59.18,8.02,39.72,37.65,JPN,Fukuoka,80.43
1,32.58,"Jeptoo, Rita",1,82.43,F,33,138.95,F1,1,NaN,...,21,5.30,NaN,99.33,16.22,69.47,65.83,KEN,Eldoret,132.10
2,16.62,"Van Dyk, Ernst F.",1,45.80,M,41,80.60,W2,1,NaN,...,1,3.08,NaN,56.45,7.75,38.03,36.10,RSA,Paarl,76.10
3,32.57,"Dibaba, Mare",3,82.43,F,24,140.58,F2,3,NaN,...,27,5.37,NaN,99.33,16.20,69.47,65.83,ETH,Shoa,132.95
4,17.12,"Hokinoue, Kota",2,46.37,M,40,81.23,W3,2,NaN,...,2,3.10,NaN,57.03,8.02,38.60,36.58,JPN,Nogata Fukuoka,76.72


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31984 entries, 0 to 31983
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   10k        31984 non-null  object 
 1   name       31984 non-null  object 
 2   division   31984 non-null  int64  
 3   25k        31984 non-null  object 
 4   gender     31984 non-null  object 
 5   age        31984 non-null  int64  
 6   official   31984 non-null  float64
 7   bib        31984 non-null  object 
 8   genderdiv  31984 non-null  int64  
 9   ctz        1244 non-null   object 
 10  35k        31984 non-null  object 
 11  overall    31984 non-null  int64  
 12  pace       31984 non-null  float64
 13  state      29408 non-null  object 
 14  30k        31984 non-null  object 
 15  5k         31984 non-null  object 
 16  half       31984 non-null  object 
 17  20k        31984 non-null  object 
 18  country    31984 non-null  object 
 19  city       31983 non-null  object 
 20  40k   

## EDA

* Detail data: 
    * Include: 31984 Rows, 21 Columns.
    * dtype: 2 float64 , 4 int64 and 15 object.
    * Missing Columns: `ctz`, `state` and `city`.
* Note: 
    * `5k, 10k,20k ,25k ,30k,35k ,40k`: run time(km).
    *  `division`: division place.(dvision winners)
    * `half` : Runner's time at halfway point
    * `overall` : ranking finishing
    * `official` : official time
    * `bib` :Assigned race number based on qualifying time.
    * `pace` : tốc độ (minute/mile)
    * `genderdiv` :thứ hạng trong giới tính
    * `ctz` : Runner's nationality 

In [20]:
# attribute unique 
df.apply(lambda x: len(x.unique()))

10k           2766
name         31915
division      6979
25k           6570
gender           2
age             64
official     11030
bib          31984
genderdiv    17575
ctz             85
35k           9516
overall      31931
pace           727
state           69
30k           8090
5k            1491
half          5651
20k           5375
country         78
city          5935
40k          10546
dtype: int64

* Inside, attribute `bib` is keys table, and các số lượng các vận động viên gần như khác nhau `name`.

In [129]:
# check dtype columns
for i in df["5k"].unique():
    try:
        float(i)
    except: 
        print(i)

-


In [130]:
df = df.replace("-",np.nan)

In [131]:
object_col = df.select_dtypes(include=['object'])
for col in object_col:
    try: 
        df[col] = df[col].astype(float)
        print(col)
    except:
        continue

10k
25k
35k
30k
5k
half
20k
40k


In [132]:
df = df.fillna(df.mean())

In [119]:
# statistic attribute
pd.set_option('float_format', '{:0.2f}'.format)
df.describe()

,10k,division,25k,age,official,genderdiv,35k,overall,pace,30k,5k,half,20k,40k
count,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00,31984.00
mean,51.80,1932.56,133.61,42.41,243.00,8051.04,196.86,15939.59,9.28,164.57,25.76,111.66,105.71,229.00
std,8.96,1715.23,25.51,11.32,52.30,4754.01,41.96,9232.98,1.99,33.93,4.32,21.04,19.76,49.13
min,16.62,1.00,45.80,18.00,80.60,1.00,67.42,1.00,3.08,56.45,7.75,38.03,36.10,76.10
25%,45.42,610.00,115.57,33.00,205.53,3972.00,167.15,7943.75,7.85,140.69,22.65,96.87,91.77,193.83
50%,50.35,1425.00,129.07,42.00,232.37,7970.00,188.22,15939.50,8.87,157.78,25.10,107.88,102.18,218.90
75%,57.15,2611.00,147.56,50.00,273.24,11968.00,220.12,23935.25,10.43,182.79,28.45,123.03,116.48,257.40
max,112.38,6979.00,289.02,81.00,538.88,17575.00,449.33,31931.00,20.57,376.38,86.95,236.67,224.35,508.35


* Nhìn chung dữ liệu numerical phân bố khá đều.
* Độ tuổi tham gia khá rộng trải dài từ 18 đến 81 tuổi.

In [101]:
df.describe(include= "O")
# silhouette_score

,name,gender,bib,ctz,state,country,city
count,31984,31984,31984,1244,29408,31984,31983
unique,31915,2,31984,84,68,78,5934
top,"Becker, Russell",M,23543,GBR,MA,USA,Boston
freq,2,17617,1,171,7587,27233,1034


* Phần lớn cuộc thi diễn ra ở Mỹ.
* Khá lớn dữ liệu thời gian bị thiếu với giá trị là `-`.
* Một số thí sinh thi 2 lần.
* Số lượng thí sinh nam nữ thi khá cân bằng.
* ...

## Preprocessing data

In [133]:
df[["ctz","state","city"]].isnull().sum(axis = 0)

ctz      30740
state     2576
city         1
dtype: int64

In [134]:
data = df.drop("ctz",axis = 1).dropna(axis = 0)
data.shape

(29408, 20)

In [135]:
object_col = data.select_dtypes(include=['object'])
object_col = object_col.drop(["name","bib"],axis = 1)

In [136]:
dummi = pd.get_dummies(object_col, prefix=object_col.columns)
dummi.head()

,gender_F,gender_M,state_AA,state_AB,state_AE,state_AK,state_AL,state_AP,state_AR,state_AZ,...,city_Yucaipa,city_Yukon,city_Zeeland,city_Zephyr Cove,city_Zephyrhills,city_Zimmerman,city_Zionsville,city_abington,city_brookline,city_woburn
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
# data.loc[:, data.columns.str.startswith("life")]
col_del = data.filter(regex='k$',axis=1).columns

In [143]:
numerical = data.select_dtypes(exclude=['object']).drop(col_del,axis =1)
numerical.head()

,division,age,official,genderdiv,overall,pace,half
9,20,31,137.83,20,20,5.27,65.23
14,13,29,87.40,13,13,3.35,40.20
15,10,30,143.90,10,46,5.50,71.68
24,14,47,89.40,14,14,3.42,41.80
27,15,30,89.52,15,15,3.42,41.82


In [144]:
data_new = pd.concat([numerical,dummi],axis = 1) 
data_new.shape

(29408, 4747)

In [146]:
t0 = time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
data_tsne = tsne.fit(data_new)
print("Execution time:",time() - t0)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 29408 samples in 21.488s...
[t-SNE] Computed neighbors for 29408 samples in 200.611s...
[t-SNE] Computed conditional probabilities for sample 1000 / 29408
[t-SNE] Computed conditional probabilities for sample 2000 / 29408
[t-SNE] Computed conditional probabilities for sample 3000 / 29408
[t-SNE] Computed conditional probabilities for sample 4000 / 29408
[t-SNE] Computed conditional probabilities for sample 5000 / 29408
[t-SNE] Computed conditional probabilities for sample 6000 / 29408
[t-SNE] Computed conditional probabilities for sample 7000 / 29408
[t-SNE] Computed conditional probabilities for sample 8000 / 29408
[t-SNE] Computed conditional probabilities for sample 9000 / 29408
[t-SNE] Computed conditional probabilities for sample 10000 / 29408
[t-SNE] Computed conditional probabilities for sample 11000 / 29408
[t-SNE] Computed conditional probabilities for sample 12000 / 29408
[t-SNE] Computed conditional probabilities for

In [150]:
tsne.shape

AttributeError: 'TSNE' object has no attribute 'shape'